In [2]:
import altair as alt
from altair import datum
import pandas as pd
import numpy as np
from vega_datasets import data

In [3]:
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [19]:
covid = pd.read_csv('data/Covid.csv')
covid.head()

,date,state,location,id,lat,lng,year,month,day,population,...,vaccine_Pfizer,vaccine_Unk_Manuf,new_vaccine_0_12,new_vaccine_12_18,new_vaccine_18_65,new_vaccine_65Plus,new_vaccine_Janssen,new_vaccine_Moderna,new_vaccine_Pfizer,new_vaccine_Unk_Manuf
0,2020-12-14,Alaska,AK,2,61.302501,-158.77502,2020,12,14,733391,...,0,0,0,0,0,0,0,0,0,0
1,2020-12-15,Alaska,AK,2,61.302501,-158.77502,2020,12,15,733391,...,0,0,0,0,0,0,0,0,0,0
2,2020-12-16,Alaska,AK,2,61.302501,-158.77502,2020,12,16,733391,...,0,0,0,0,0,0,0,0,0,0
3,2020-12-17,Alaska,AK,2,61.302501,-158.77502,2020,12,17,733391,...,2,0,0,0,0,0,0,0,2,0
4,2020-12-18,Alaska,AK,2,61.302501,-158.77502,2020,12,18,733391,...,2,0,0,0,0,0,0,0,0,0


In [20]:
covid['date'] = pd.to_datetime(covid['date'])

In [12]:
year = 2022
month = 3
day = 10

In [25]:
df1 = pd.read_csv('data/df1.csv')
df1['date'] = pd.to_datetime(df1['date'])

In [26]:
states = alt.topo_feature(data.us_10m.url, 'states')

vis1_selection = alt.selection_single(fields=["location"])

vis1_colorCondition1 = alt.condition(vis1_selection, 
                                        alt.Color('covid_rate:Q', 
                                                    scale=alt.Scale(scheme='reds'), 
                                                    legend=alt.Legend(title='COVID case rate')), 
                                        alt.value("gray"))
vis1_colorCondition2 = alt.condition(vis1_selection, 
                                        alt.Color('avg_vaccine:Q', 
                                                    scale=alt.Scale(scheme='blues'), 
                                                    legend=alt.Legend(title='Avg vaccine dose')), 
                                        alt.value("gray"))

vis1_base = alt.Chart(df1).mark_geoshape().transform_lookup(
    lookup='id',
    from_=alt.LookupData(states, 'id', fields=["type", "properties", "geometry"])
).transform_filter(
    datum.year == year & datum.month == month & datum.day == day
).transform_calculate(
    covid_rate='datum.tot_case / datum.population',
    avg_vaccine='datum.tot_vaccine / datum.population'
).encode(
    tooltip=["state:N", "covid_rate:Q", "avg_vaccine:Q"]
).project(
    type='albersUsa'
).add_selection(
    vis1_selection
)

vis1_map1 = vis1_base.encode(
    color=vis1_colorCondition1
)

vis1_map2 = vis1_base.encode(
    color=vis1_colorCondition2
)

(vis1_map1 & vis1_map2).resolve_scale(
    color='independent'
).properties(
    title='COVID case rate vs. average vaccine dose by state'
)

alt.VConcatChart(...)

In [8]:
df2 = pd.read_csv('data/df2.csv')
df2['date'] = pd.to_datetime(df2['date'])

In [28]:
age = ['All', 'age<12', '12<=age<18', '18<=age<65', 'age>=65']
type = ['All', 'Janssen', 'Moderna', 'Pfizer', 'Unknown']

vis2_selection1 = alt.selection_multi(fields=['vaccine_type'], bind='legend', empty="none")

vis2_selection2 = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['week_date'], empty='none')
vis2_selection3=alt.selection_interval(bind="scales", encodings=["x"])

vis2_base = alt.Chart(df2).transform_filter(
    datum.year <= year & datum.month <= month & datum.day <= day
).transform_filter(
    alt.FieldOneOfPredicate(field='vaccine_type', oneOf=type)
).transform_aggregate(
    sum_case='sum(new_case)',
    sum_vaccine='sum(new_vaccine)',
    week_date='max(date)',
    groupby=['vaccine_type', 'year', 'week']
).encode(
    x=alt.X('week_date:T', title='Date')
)      

vis2_line1 = vis2_base.mark_line(size=3).encode(
    y=alt.Y('sum_case:Q', title='New COVID case (solid)'),
    color=alt.value("red")
)

vis2_line2 = vis2_base.mark_line(size=3).encode(
    y=alt.Y('sum_vaccine:Q', title='New administered vaccine (dashed)'),
    strokeDash=alt.value([5,5]),
    color=alt.Color('vaccine_type', scale=alt.Scale(scheme='dark2'), title='Vaccine type'),
    opacity=alt.condition(vis2_selection1, alt.value(1), alt.value(0.2))
).add_selection(
    vis2_selection1
)

vis2_selectors = vis2_base.mark_point().encode(
    opacity=alt.value(0),
).add_selection(
    vis2_selection2
)

vis2_points1 = vis2_base.mark_point(size=80, filled=True).encode(
    y=alt.Y('sum_case:Q', axis=None),
    opacity=alt.condition(vis2_selection2, alt.value(1), alt.value(0)),
    tooltip=['week_date:T', 'sum_case:Q'],
    color=alt.value("red")
)

vis2_points2 = vis2_base.mark_point(size=80, filled=True).encode(
    y=alt.Y('sum_vaccine:Q', axis=None),
    color=alt.Color('vaccine_type', scale=alt.Scale(scheme='dark2'), title='Vaccine type'),
    opacity=alt.condition(vis2_selection2 & vis2_selection1, alt.value(1), alt.value(0)),
    tooltip=['week_date:T', 'vaccine_type:N', 'sum_vaccine:Q']
)

vis2_rules = vis2_base.mark_rule(color='gray', size=2).transform_filter(
    vis2_selection2
)

alt.layer(
    vis2_line1, vis2_line2, vis2_selectors, vis2_points1, vis2_points2, vis2_rules
).add_selection(
    vis2_selection3
).resolve_scale(
    y='independent'
).properties(
    width=600, height=300
)

alt.LayerChart(...)